In [0]:
print("hello")

hello


In [0]:
import pyspark
sc = pyspark.SparkContext()
sqlContext = pyspark.SQLContext(sc)

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<command-2194105566022078> in <cell line: 2>()
      1 import pyspark
----> 2 sc = pyspark.SparkContext()
      3 sqlContext = pyspark.SQLContext(sc)

/databricks/spark/python/pyspark/context.py in __init__(self, master, appName, sparkHome, pyFiles, environment, batchSize, serializer, conf, gateway, jsc, profiler_cls, udf_profiler_cls)
    197             )
    198 
--> 199         SparkContext._ensure_initialized(self, gateway=gateway, conf=conf)
    200         try:
    201             self._do_init(

/databricks/spark/python/pyspark/context.py in _ensure_initialized(cls, instance, gateway, conf)
    476 
    477                     # Raise error if there is already a running Spark context
--> 478                     raise ValueError(
    479                         "Cannot run multiple SparkContexts at once; "
    480                

In [0]:
df = sc.textFile("/FileStore//tables/2015_12_12.csv")

In [0]:
df

Out[14]: /FileStore//tables/2015_12_12.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [0]:
df.take(10)

Out[15]: ['"date","time","size","r_version","r_arch","r_os","package","version","country","ip_id"',
 '"2015-12-12","13:42:10",257886,"3.2.2","i386","mingw32","HistData","0.7-6","CZ",1',
 '"2015-12-12","13:24:37",1236751,"3.2.2","x86_64","mingw32","RJSONIO","1.3-0","DE",2',
 '"2015-12-12","13:42:35",2077876,"3.2.2","i386","mingw32","UsingR","2.0-5","CZ",1',
 '"2015-12-12","13:42:01",266724,"3.2.2","i386","mingw32","gridExtra","2.0.0","CZ",1',
 '"2015-12-12","13:00:21",3687766,NA,NA,NA,"lme4","1.1-10","DE",3',
 '"2015-12-12","13:08:56",57429,NA,NA,NA,"testthat","0.11.0","DE",3',
 '"2015-12-12","13:08:09",216068,"3.2.2","x86_64","mingw32","mvtnorm","1.0-3","DE",4',
 '"2015-12-12","13:25:00",3595497,"3.2.2","x86_64","mingw32","maps","3.0.1","DE",2',
 '"2015-12-12","13:25:05",1579597,"3.2.2","x86_64","mingw32","sp","1.2-1","DE",2']

In [0]:
df.count()

Out[16]: 421970

In [0]:
df.takeSample(True, 5, 3)

Out[17]: ['"2015-12-12","05:59:49",511365,"3.2.3","i386","mingw32","ks","1.10.0","US",10364',
 '"2015-12-12","05:10:24",287843,"3.2.2","x86_64","mingw32","xtable","1.8-0","US",5665',
 '"2015-12-12","13:06:32",494138,"3.2.3","x86_64","linux-gnu","rjson","0.2.15","KR",655',
 '"2015-12-12","23:29:36",1350975,"3.2.3","x86_64","mingw32","randtoolbox","1.17","US",6509',
 '"2015-12-12","13:13:10",45187,"3.2.3","x86_64","linux-gnu","spatial","7.3-11","GB",548']

In [0]:
#We may need to note that each row of the data is a character string, and it would be more convenient to have an array in some senarios. We can use map to transform them and use take method to get the first a few rows to check how the resutls look like.

df1=df.map(lambda x : x.split(','))
df1.take(3)

Out[19]: [['"date"',
  '"time"',
  '"size"',
  '"r_version"',
  '"r_arch"',
  '"r_os"',
  '"package"',
  '"version"',
  '"country"',
  '"ip_id"'],
 ['"2015-12-12"',
  '"13:42:10"',
  '257886',
  '"3.2.2"',
  '"i386"',
  '"mingw32"',
  '"HistData"',
  '"0.7-6"',
  '"CZ"',
  '1'],
 ['"2015-12-12"',
  '"13:24:37"',
  '1236751',
  '"3.2.2"',
  '"x86_64"',
  '"mingw32"',
  '"RJSONIO"',
  '"1.3-0"',
  '"DE"',
  '2']]

In [0]:
def clean(x):
    return[xx.replace('"','') for xx in x]
df1=df1.map(clean)
df1.take(3)

Out[21]: [['date',
  'time',
  'size',
  'r_version',
  'r_arch',
  'r_os',
  'package',
  'version',
  'country',
  'ip_id'],
 ['2015-12-12',
  '13:42:10',
  '257886',
  '3.2.2',
  'i386',
  'mingw32',
  'HistData',
  '0.7-6',
  'CZ',
  '1'],
 ['2015-12-12',
  '13:24:37',
  '1236751',
  '3.2.2',
  'x86_64',
  'mingw32',
  'RJSONIO',
  '1.3-0',
  'DE',
  '2']]

In [0]:
package_count= df1.map(lambda x : (x[6], 1)).reduceByKey(lambda a,b: a+b)
package_count.take(5)

Out[27]: [('HistData', 159),
 ('UsingR', 151),
 ('lme4', 1560),
 ('testthat', 1178),
 ('maps', 1586)]

In [0]:
package_count.count()

Out[28]: 8660

In [0]:
pc2=df1.map(lambda x : (x[6],1)).countByKey()


---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<command-2194105566022089> in <cell line: 2>()
      1 pc2=df1.map(lambda x : (x[6],1)).countByKey()
----> 2 pc2.take(5)

AttributeError: 'collections.defaultdict' object has no attribute 'take'

In [0]:
pc2['HistData']

Out[31]: 159

In [0]:
#sort in descending order
package_count.map(lambda x : (x[1],x[0])).sortByKey(0).take(10)

Out[36]: [(4783, 'Rcpp'),
 (3913, 'ggplot2'),
 (3748, 'stringi'),
 (3449, 'stringr'),
 (3436, 'plyr'),
 (3265, 'magrittr'),
 (3223, 'digest'),
 (3205, 'reshape2'),
 (3046, 'RColorBrewer'),
 (3007, 'scales')]

In [0]:
#sort in ascending order
package_count.map(lambda x : (x[1],x[0])).sortByKey(1).take(100)

Out[42]: [(1, 'multic'),
 (1, 'RBerkeley'),
 (1, 'vimcom'),
 (1, 'waldwolf'),
 (1, 'bstats'),
 (1, 'parspatstat'),
 (1, 'WaveCGH'),
 (1, 'mixnet'),
 (1, 'postgwas'),
 (1, 'rolasized'),
 (1, 'ropensnp'),
 (1, 'roxyPackage'),
 (1, 'rImpactStory'),
 (1, 'rcqp'),
 (1, 'skewtools'),
 (1, 'spclust'),
 (1, 'ARAMIS'),
 (1, 'CVD'),
 (1, 'ROracleUI'),
 (1, 'mr'),
 (1, 'SSOAP'),
 (1, 'paleoTSalt'),
 (1, 'PARccs'),
 (1, 'ris'),
 (1, 'TradeStrategyAnalyzer'),
 (1, 'swst'),
 (1, 'ChangeAnomalyDetection'),
 (1, 'EDanalysis'),
 (1, 'EasyUpliftTree'),
 (1, 'RAFM'),
 (1, 'gcbd'),
 (1, 'HTSDiff'),
 (1, 'GetNenshuJP'),
 (1, 'MASSI'),
 (1, 'MiClip'),
 (1, 'Reot'),
 (1, 'SCEPtERextras'),
 (1, 'SDBP'),
 (1, 'TSjson'),
 (1, 'TCC'),
 (1, 'cldr'),
 (1, 'easystab'),
 (1, 'emudata'),
 (1, 'igraph0'),
 (1, 'DART'),
 (1, 'PACE'),
 (1, 'graphite'),
 (1, 'iScreen'),
 (1, 'logistiX'),
 (1, 'littler'),
 (1, 'ieeeround'),
 (1, 'em2'),
 (1, 'nfda'),
 (1, 'mpc'),
 (1, 'oosp'),
 (1, 'RGtk2DfEdit'),
 (1, 'cxxPack'),
 (1, 'p

In [0]:
package_count.sortBy(lambda x:x[1]).take(5)  # default ascending is True

Out[39]: [('multic', 1),
 ('RBerkeley', 1),
 ('vimcom', 1),
 ('waldwolf', 1),
 ('bstats', 1)]

In [0]:
package_count.sortBy(lambda x:x[1], ascending= False).take(5)  # default ascending is True

Out[40]: [('Rcpp', 4783),
 ('ggplot2', 3913),
 ('stringi', 3748),
 ('stringr', 3449),
 ('plyr', 3436)]

In [0]:
df1.filter(lambda x : x[6] =='ggplot2' and x[8] == 'CN').count()

Out[46]: 172

In [0]:
df1.filter(lambda x : x[6] =='ggplot2' and x[8] == 'CN').take(5)

Out[48]: [['2015-12-12',
  '03:38:38',
  '2677791',
  '3.2.2',
  'x86_64',
  'mingw32',
  'ggplot2',
  '1.0.1',
  'CN',
  '347'],
 ['2015-12-12',
  '03:42:26',
  '2677787',
  '3.0.2',
  'i386',
  'mingw32',
  'ggplot2',
  '1.0.1',
  'CN',
  '356'],
 ['2015-12-12',
  '03:46:06',
  '2677787',
  '3.2.2',
  'x86_64',
  'mingw32',
  'ggplot2',
  '1.0.1',
  'CN',
  '463'],
 ['2015-12-12',
  '03:46:54',
  '160869',
  '3.2.3',
  'x86_64',
  'mingw32',
  'ggplot2',
  '1.0.1',
  'CN',
  '521'],
 ['2015-12-12',
  '08:47:37',
  '2677790',
  '3.2.2',
  'i386',
  'mingw32',
  'ggplot2',
  '1.0.1',
  'CN',
  '816']]

In [0]:
temp= df1.filter(lambda x : x[6] == 'Rtts' and x[8] == 'CN').collect()

In [0]:
temp

Out[50]: [['2015-12-12',
  '20:15:24',
  '23820',
  '3.2.2',
  'x86_64',
  'mingw32',
  'Rtts',
  '0.3.3',
  'CN',
  '41']]

In [0]:
df.count()

Out[51]: 421970

In [0]:
df.union(df).count()

Out[52]: 843940

In [0]:
df.intersection(df).count()

Out[53]: 421553

In [0]:
content_modified=df.map(lambda x:(x[8], x))

In [0]:
mapping=[('DE', 'Germany'), ('US', 'United States'), ('CN', 'China'), ('IN',"India")]
mapping=sc.parallelize(mapping)

In [0]:
content_modified.join(mapping).takeSample(False, 8)


Out[68]: []

In [0]:
type(mapping)

Out[66]: pyspark.rdd.RDD

In [0]:
content_modified.leftOuterJoin(mapping).takeSample(False, 8)

Out[67]: [('-',
  ('"2015-12-12","16:19:26",1788332,"3.2.3","x86_64","linux-gnu","viridis","0.3.1","CN",16573',
   None)),
 ('-',
  ('"2015-12-12","13:14:36",180621,"3.2.3","x86_64","linux-gnu","yaml","2.1.13","RU",1410',
   None)),
 ('-',
  ('"2015-12-12","20:08:56",35214,NA,NA,NA,"reshape2","1.4.1","ES",16753',
   None)),
 ('-',
  ('"2015-12-12","16:41:42",185782,"3.2.2","x86_64","mingw32","Shrinkage","1.0","KR",4989',
   None)),
 ('-',
  ('"2015-12-12","20:31:39",361917,"3.2.3","x86_64","linux-gnu","wmtsa","2.0-0","GB",5315',
   None)),
 ('-',
  ('"2015-12-12","08:07:39",382711,"3.2.2","x86_64","mingw32","httr","1.0.0","KR",823',
   None)),
 ('-',
  ('"2015-12-12","17:37:26",626342,"3.2.3","x86_64","mingw32","roxygen2","5.0.1","NL",3072',
   None)),
 ('-',
  ('"2015-12-12","18:38:36",23590,"3.0.3","x86_64","mingw32","learNN","0.2.0","CN",41',
   None))]

In [0]:
content_modified.rightOuterJoin(mapping).takeSample(False, 8)

Out[69]: [('US', (None, 'United States')),
 ('CN', (None, 'China')),
 ('DE', (None, 'Germany')),
 ('IN', (None, 'India'))]

In [0]:
df1.cache()

Out[74]: PythonRDD[140] at RDD at PythonRDD.scala:58

In [0]:
df1.is_cached

Out[80]: False

In [0]:
df1.unpersist()

Out[79]: PythonRDD[140] at RDD at PythonRDD.scala:58